In [1]:
from __future__ import print_function
import os
import sys
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from six.moves import cPickle as pickle
%matplotlib inline

In [2]:
#load the dataset from pickle file
fileName='Image_data.pickle'
with open(fileName,'rb') as f:
    data=pickle.load(f)
    train_dataset=data['train_dataset']
    train_labels=data['train_labels']
    
    valid_dataset=data['valid_dataset']
    valid_labels=data['valid_labels']
    
    test_dataset=data['test_dataset']
    test_labels=data['test_labels']
    
    del(data)
    f.close()

In [3]:
#reformat the data into 4-D tensor
cloth_channels=3
image_size=28
cloth_labels=2

def reformat_Data(dataset,labels):
    dataset=dataset.reshape((-1,image_size,image_size,cloth_channels)).astype(float)
    labels=(np.arange(cloth_labels)==labels[:,None]).astype(float)
    return dataset,labels

In [4]:
train_dataset,train_labels=reformat_Data(train_dataset,train_labels)

test_dataset,test_labels=reformat_Data(test_dataset,test_labels)

valid_dataset,valid_labels=reformat_Data(valid_dataset,valid_labels)

In [6]:
#initialize
batch_size=16
patch_size=5
depth=4
more_depth=5
num_hidden=6
image_size=28


graph=tf.Graph()
with graph.as_default():
    tf_train_dataset=tf.placeholder(tf.float32,shape=(batch_size,image_size,image_size,cloth_channels))
    tf_train_labels=tf.placeholder(tf.float32,shape=(batch_size,cloth_labels))
    tf_valid_dataset=tf.constant(valid_dataset)
    tf_test_dataset=tf.constant(test_dataset)
    
    # Weight and Bias Parameters
    K=2
    ###### Layer 1
    w1=tf.Variable(tf.truncated_normal(shape=[patch_size,patch_size,cloth_channels,K],stddev=0.1)) ##### W[5,5,1,4] ->[,28,28,4]
    b1=tf.Variable(tf.zeros([K]))
    
    ###### Layer 2
    w2=tf.Variable(tf.truncated_normal(shape=[patch_size,patch_size,K,depth],stddev=0.1))  ##### W[5,5,4,16] ->[,14,14,16]
    b2=tf.Variable(tf.constant(1.0,shape=[depth]))
    
    ###### Layer 3
    w3=tf.Variable(tf.truncated_normal(shape=[patch_size,patch_size,depth,more_depth],stddev=0.1))  ###### W[5,5,16,16] ->[,7,7,16]
    b3=tf.Variable(tf.constant(1.0,shape=[more_depth]))
    
    ###### connected Layer
    w4=tf.Variable(tf.truncated_normal(shape=[image_size//4 * image_size//4 * more_depth,num_hidden],stddev=0.1)) ##### W[7*7*16,64] ->[,64]
    b4=tf.Variable(tf.constant(1.0,shape=[num_hidden]))
    
    ###### final layer
    w5=tf.Variable(tf.truncated_normal(shape=[num_hidden,cloth_labels],stddev=0.1)) ####### W[64,10]
    b5=tf.Variable(tf.constant(1.0, shape=[cloth_labels]))
    
    
    # Computing CNN model
    def model(data):
        conv1=tf.nn.relu(tf.nn.conv2d(data,w1, strides=[1,1,1,1],padding='SAME') +b1)
        
        conv2=tf.nn.relu(tf.nn.conv2d(conv1,w2, strides=[1,2,2,1], padding='SAME') +b2)
        
        conv3=tf.nn.relu(tf.nn.conv2d(conv2,w3, strides=[1,2,2,1], padding='SAME') +b3)
        
        conv3_shape=conv3.get_shape().as_list()
        conv3_reshape=tf.reshape(conv3, [conv3_shape[0],conv3_shape[1]*conv3_shape[2]*conv3_shape[3]])
        
        #connected Layer
        connec=tf.nn.relu(tf.matmul(conv3_reshape,w4) +b4)
        
        return (tf.matmul(connec,w5) +b5)
    
    # Train Computation
    logits=model(tf.cast(tf_train_dataset,dtype=tf.float32))
    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    
    # Optimizer
    optimizer=tf.train.GradientDescentOptimizer(0.005).minimize(loss)
    
    #Predictions
    train_prediction=tf.nn.softmax(logits)
    test_prediction=tf.nn.softmax(model(tf.cast(tf_test_dataset,dtype=tf.float32)))
    valid_prediction=tf.nn.softmax(model(tf.cast(tf_valid_dataset,dtype=tf.float32)))

In [7]:
#Accuracy

def accuracy(predictions,labels):
    return (100.0*np.sum(np.argmax(predictions,1)==np.argmax(labels,1))/predictions.shape[0])

In [9]:
# Iterate the model and train 
num_steps=101

    
with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    print("Initialised......")
    saver=tf.train.Saver()
    for step in range(num_steps):
        offset=np.random.randint(0,(45-batch_size))
        
        batch_data=train_dataset[offset:(offset+batch_size),:,:,:]
        batch_labels=train_labels[offset:(offset+batch_size),:]
        
        feed_dict={tf_train_dataset:batch_data, tf_train_labels:batch_labels}
        opt,l,pred=sess.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        if (step%10==0):
            print("loss at %d:%f" %(step,l))
            print("Mini Batch Accuracy:%.2f%%" %accuracy(pred,batch_labels))
            print("Validation Accuracy:%.2f%%" %accuracy(valid_prediction.eval(),valid_labels))
    print("-"*30)
    
    print("Test Accuracy:%.2f%%" %accuracy(test_prediction.eval(),test_labels))
    saver.save(sess,'image-classify')

Initialised......
loss at 0:0.960424
Mini Batch Accuracy:25.00%
Validation Accuracy:30.00%
loss at 10:0.698405
Mini Batch Accuracy:31.25%
Validation Accuracy:70.00%
loss at 20:0.623455
Mini Batch Accuracy:75.00%
Validation Accuracy:70.00%
loss at 30:0.587565
Mini Batch Accuracy:75.00%
Validation Accuracy:70.00%
loss at 40:0.569815
Mini Batch Accuracy:75.00%
Validation Accuracy:70.00%
loss at 50:0.560917
Mini Batch Accuracy:75.00%
Validation Accuracy:70.00%
loss at 60:0.561982
Mini Batch Accuracy:75.00%
Validation Accuracy:70.00%
loss at 70:0.559851
Mini Batch Accuracy:75.00%
Validation Accuracy:70.00%
loss at 80:0.619838
Mini Batch Accuracy:68.75%
Validation Accuracy:70.00%
loss at 90:0.685327
Mini Batch Accuracy:62.50%
Validation Accuracy:70.00%
loss at 100:0.617018
Mini Batch Accuracy:68.75%
Validation Accuracy:70.00%
------------------------------
Test Accuracy:71.43%
